# Proyecto : **DATA CLEANING Y GENERADOR DE UN MAPA DE CALOR**
***

Importar librerías

In [1]:
import folium, re
import pandas as pd, geopandas as gpd, leafmap.foliumap as leafmap
from folium.plugins import HeatMap
from estilos import *
from shapely.geometry import Point


Definimos la ruta de los archivos con los que trabajaremos

In [2]:
# TABLAS
archivo_excel = "COMPILADO-MARZO.xlsx"

# SHP
nodos_shp = 'SHP/NODOS.shp'
cables_shp = 'SHP/enlaces_logicos.shp'
distritos = "shp/distritos.shp"

## **1. Data cleaning**
***

Las siguientes líneas de código tienen como objetivo corregir errores e inexactitudes en los datos del reporte, garantizando que sean coherentes y precisos. Nos enfocamos en mejorar los valores de las **coordenadas** mediante los siguientes pasos:.

1. Convierte todas las coordenadas a grados decimales.
2. Ordena correctamente los valores en las columnas correctas.
3. Marca las celdas vacías con "S/D".
4. Crea una columna "Estado" con "OK" si ambas coordenadas están presentes.

> *El archivo de entrada será un excel con coordendas y el resultado se guardará en un archivo con el mismo formato.*

In [3]:
# Función para limpiar valores (eliminar espacios y comas)
def limpiar_valor(valor):
    if pd.isna(valor) or str(valor).strip() == "":  # Si está vacío, devolver "S/D"
        return "S/D"
    valor = str(valor).strip().replace(',', '')  # Eliminar espacios y comas
    return valor if valor else "S/D"

# Función para convertir coordenadas a grados decimales
def convertir_a_decimal(coordenada):
    if coordenada == "S/D":
        return coordenada
    try:
        # Buscar si está en formato grados, minutos y segundos (GMS)
        match = re.match(r'(\d+)[°º]?\s*(\d*)\'?\s*(\d*\.?\d*)"?\s*([NSOE])?', coordenada)
        if match:
            grados, minutos, segundos, direccion = match.groups()
            grados = float(grados)
            minutos = float(minutos) if minutos else 0
            segundos = float(segundos) if segundos else 0

            decimal = grados + (minutos / 60) + (segundos / 3600)
            if direccion in ['S', 'O']:  # Sur y Oeste son negativos
                decimal *= -1
            return round(decimal, 6)
        
        # Si ya está en decimal, convertir a float
        return round(float(coordenada), 6)
    
    except ValueError:
        return "S/D"

# Función para verificar si un valor es latitud o longitud en Perú
def identificar_lat_lon(lat, lon):
    if lat == "S/D" or lon == "S/D":
        return lat, lon  # Si falta un valor, devolverlo tal cual
    
    lat, lon = float(lat), float(lon)
    
    # Rango de Perú: latitud (-18 a 0), longitud (-82 a -68)
    if -18 <= lat <= 0 and -82 <= lon <= -68:
        return lat, lon
    elif -18 <= lon <= 0 and -82 <= lat <= -68:  # Si están invertidos
        return lon, lat
    else:
        return "S/D", "S/D"  # Coordenadas fuera del rango esperado

# Cargar el archivo Excel
df = pd.read_excel(archivo_excel)

# Limpiar los valores de latitud y longitud
df["Latitud"] = df["Latitud"].apply(limpiar_valor)
df["Longitud"] = df["Longitud"].apply(limpiar_valor)

# Convertir a grados decimales
df["Latitud"] = df["Latitud"].apply(convertir_a_decimal)
df["Longitud"] = df["Longitud"].apply(convertir_a_decimal)

# Corregir valores de latitud y longitud
df[["Latitud", "Longitud"]] = df.apply(lambda row: identificar_lat_lon(row["Latitud"], row["Longitud"]), axis=1, result_type="expand")

# Crear columna "Estado": OK si ambas coordenadas están presentes
df["Estado"] = df.apply(lambda row: "OK" if row["Latitud"] != "S/D" and row["Longitud"] != "S/D" else "S/D", axis=1)

# Guardar el archivo corregido
df.to_excel("DATA_CORREGIDA.xlsx", index=False)

print("Proceso finalizado. Se ha generado 'DATA_CORREGIDA.xlsx'.")

Proceso finalizado. Se ha generado 'DATA_CORREGIDA.xlsx'.


Ahora validamos manualmente los valore sgenerados y levantamos alguna observación al respecto.

**Volvemos a importar nuestro archivo de coordendas limpias**

In [7]:
df_data = pd.read_excel("DATA_CORREGIDA.xlsx")


# Definir los distritos de las sedes
# 1. Cargar el archivo SHP con los distritos
distritos2 = gpd.read_file(distritos)

# 2. Convertir las coordenadas en puntos geográficos
geometry = [Point(xy) for xy in zip(df_data["Longitud"], df_data["Latitud"])]
gdf_coordenadas = gpd.GeoDataFrame(df_data, geometry=geometry, crs=distritos2.crs)

# 3. Realizar la intersección espacial para asignar distrito
gdf_resultado = gpd.sjoin(gdf_coordenadas, distritos2, how="left", predicate="within")

# 4. Eliminar columnas que no necesitamos
df = gdf_resultado.drop(columns=['geometry', 'index_right'])  # Reemplaza con los nombres de tu

# Guardar el archivo corregido
df.to_excel("DATA_CORREGIDA_REGIONES.xlsx", index=False)

df.head()

,ID,Código,Cliente,Consultor,Dirección,Cabecera,Creado el,MES,Estado,Plazo (días),...,Cerrado el,SLA,SEGMENTO,Longitud,Latitud,Monto Total Factibilidad,Tipo de Factibilidad,DEPARTAMEN,PROVINCIA,DISTRITO
0,ENE673,NaN,Innova School,Maribel Carpio,"Av. Los Ángeles, Mz. B Lt. 6, sector Chorrillo...",Principal,45685.00000,ENERO,OK,7.0,...,45687.000000,SI,Corporaciones,-70.223112,-17.994502,9931.6,MASIVO,TACNA,TACNA,POCOLLAY
1,MAR163,NaN,DOMINION PERÚ,Helga Medina,MZA. D LOTE. 11 URB. ZONA AUXILIAR PARQUE INDU...,NaN,45729.00000,MARZO,OK,NaN,...,45733.000000,SI,Mayorista,-70.237239,-17.990761,14090.14,Masivo,TACNA,TACNA,ALTO DE LA ALIANZA
2,ENE278,NaN,TINKA,Abel Elias,"TDA ILO MZ 47 S/N, ILO | TX",Principal,45671.00000,ENERO,OK,8.0,...,45679.000000,SI,Corporaciones,-71.329097,-17.651000,NaN,MASIVO,MOQUEGUA,ILO,ILO
3,ENE742,NaN,APUESTA TOTAL,Pablo MOlina,Calle Ilo N° 419 Dpto 102,Principal,45688.00000,ENERO,OK,7.0,...,45688.000000,SI,Corporaciones,-71.344456,-17.647583,5689,MASIVO,MOQUEGUA,ILO,ILO
4,MAR530,61208.0,ENAPU S.A - 20100003199 - EMP NACIONAL DE PUER...,Frida Tejada,CALLE matara 104 ENTIDAD PUBLICA LOCAL,Principal,45733.75059,MARZO,OK,2.0,...,45734.738831,SI,Gobierno,-71.346845,-17.647009,52062,Intranet,MOQUEGUA,ILO,ILO


## **2. Generar un mapa de calor interactivo**
***

Las siguientes líneas de código tienen como objetivo generar un mapa de calor utilizando los datos procesados anteriormente. Al finalizar, se presenta el resultado en un formato interactivo, acompañado de otros elementos de red que enriquecen la presentación.

> *El formato de salida es un HTML.*

In [ ]:
mapa = leafmap.Map(
    center=[df["Latitud"].mean(), df["Longitud"].mean()], 
    zoom=5.5,
    tiles= "Esri.WorldImagery")



#Creamos los controles por capas
Calor = folium.FeatureGroup("Heatmap").add_to(mapa)
factis = folium.FeatureGroup("Factibilidades").add_to(mapa)
Nodos = folium.FeatureGroup("Nodos").add_to(mapa)
Cables = folium.FeatureGroup("Cables").add_to(mapa)


# Agregar puntos al mapa

estilo_puntos = {"color": "darkblue", "fill_color": "skyblue", "radius": 5}
# Factibilidades atendidas
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=estilo_puntos.get("radius", 5),
        color=estilo_puntos.get("color", "blue"),
        fill=True,
        fill_opacity = 0.7,
        fill_color=estilo_puntos.get("fill_color", "blue"),
        popup= "ID: "
            + str(row["ID"])
            + "<br>"
            + "Cliente: "
            + str(row["Cliente"])
            + "<br>"
            + "Costo: S/."
            + str(row["MONTO TOTAL FACTIBLE"])
    ).add_to(factis)

# Leer cables
gdf = gpd.read_file(cables_shp)
folium.GeoJson(gdf,
                   style_function=lambda feature: {
                        "color": "#55ff00",  # Cambia el color aquí
                        "weight": 2,       # Grosor de la línea
                    }).add_to(Cables)

# Leer nodo
gdf = gpd.read_file(nodos_shp)
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(color="#ffaa00", icon="info-sign")
    ).add_to(Nodos)


# Agregar capa de mapa de calor
HeatMap(df[['Latitud', 'Longitud']].values, radius=15).add_to(Calor)


# Define the legend
legend_dict = {
    "Nodos": "#D43D2A",
    "Factibilidades atendidas": "Darkblue",
    "Cables": "#55ff00"
}

mapa.add_legend(title="Elementos", legend_dict=legend_dict)


# Agregar título
titulo = folium.Element(html)
mapa.get_root().html.add_child(titulo)

folium.LayerControl(collapsed=False).add_to(mapa) #'topleft',


# Guardar el mapa en un archivo HTML
mapa.save(f"Mapa de calor - {input()}.html")
mapa